In [1]:
# -*- coding: utf-8 -*-
from __future__ import print_function
from __future__ import division
import datetime

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import pandas as pd
import numpy as np

In [3]:
files = [('../data/raw/Baen4185_ODBA3D_c.csv', '../data/raw/Baen4185_ODBB.csv'),
         ('../data/raw/Dorje4014_ODBA3D_c.csv', '../data/raw/Dorje4014_ODBB.csv')]

In [20]:
def process_file_tuple(file_tuple):
    dfraw1 = pd.read_csv(file_tuple[0], parse_dates=[['Day', 'Time']], na_values=[' '])
    dfraw1.set_index('EventID', inplace=True)
    print('%d rows read from %s' % (len(dfraw1), file_tuple[0]))
    dup = dfraw1.index.duplicated(keep='first')
    dfraw1 = dfraw1[dup==False]
    print('%d duplicate indices removed, %d records remaining' % (dup.sum(), len(dfraw1)))
    dfraw2 = pd.read_csv(file_tuple[1], parse_dates=[['Day', 'Time']], na_values=[' '])
    dfraw2.set_index('EventID', inplace=True)
    print('%d rows read from %s' % (len(dfraw2), file_tuple[1]))
    dup = dfraw2.index.duplicated(keep='first')
    dfraw2 = dfraw2[dup==False]
    print('%d duplicate indices removed, %d records remaining' % (dup.sum(), len(dfraw2)))
    dfraw = dfraw1.join(dfraw2, how='inner', rsuffix='_dup')
    print('Inner-joined dataframes on index, %d records remaining' % len(dfraw))
    df = pd.DataFrame(dfraw.index)
    df.set_index('EventID', inplace=True)
    df['dt'] = dfraw['Day_Time']
    df['odba'] = dfraw['odba']
    acc_x = data=dfraw[[col for col in dfraw.columns if col.startswith('ACCX')]].values
    acc_y = data=dfraw[[col for col in dfraw.columns if col.startswith('ACCY')]].values
    acc_z = data=dfraw[[col for col in dfraw.columns if col.startswith('ACCZ')]].values
    df['groundspeed'] = dfraw['Groundspeed']
    df['pressuredelta'] = dfraw['PressureVerticalVelocity']
    df['vvelocity'] = dfraw['VVelocity']
    df['uvelocity'] = dfraw['UVelocity']
    altitude = dfraw.HeightMSL-dfraw.SRTMElevation  # height above MSL - SRTM elevation = actual altitude?
    altitude[altitude < 0] = 0.0        # normalising erroneous altitudes 
    altitude[altitude > 1000] = 1000.0  # (too high or too low)
    df['altitude'] = altitude
    temperature = dfraw['Temperature']-273.15  # in Celsius
    temperature[temperature < -20] = -20.0 # normalising erroneous temperatures
    temperature[temperature > 50] = 50.0 # normalising erroneous temperatures
    df['temperature'] = temperature
    df['humidity'] = dfraw['RelativeHumidity']
    try:
        df['state'] = dfraw['Behavior']
    except KeyError:
        df['state'] = dfraw['Behaviour']
    
    # processing acceleration data
    # purging records that do not contain 40 acceleration values on each axis
    invalid = np.apply_along_axis(lambda x: np.any(np.isnan(x)), 1, acc_x) \
              | np.apply_along_axis(lambda x: np.any(np.isnan(x)), 1, acc_y) \
              | np.apply_along_axis(lambda x: np.any(np.isnan(x)), 1, acc_z)
    if invalid.any():
        df = df[invalid==False]
        acc_x = acc_x[invalid==False]
        acc_y = acc_y[invalid==False]
        acc_z = acc_z[invalid==False]
        print('Purged %d records with less than 40 acc values on at least 1 axis, %d remaining' % (invalid.sum(), len(df)))
    # standardising all series to mean 0 and std 1
    acc_x = (acc_x-(acc_x.mean()))/(acc_x.std())
    acc_y = (acc_y-(acc_y.mean()))/(acc_y.std())
    acc_z = (acc_z-(acc_z.mean()))/(acc_z.std())
    
    # creating composite measures for each acceleration burst: mean and std
    df['acc_x_mean'] = np.apply_along_axis(lambda x: x.mean(), 1, acc_x)
    df['acc_x_std'] = np.apply_along_axis(lambda x: x.std(), 1, acc_x)
    df['acc_y_mean'] = np.apply_along_axis(lambda x: x.mean(), 1, acc_y)
    df['acc_y_std'] = np.apply_along_axis(lambda x: x.std(), 1, acc_y)
    df['acc_z_mean'] = np.apply_along_axis(lambda x: x.mean(), 1, acc_z)
    df['acc_z_std'] = np.apply_along_axis(lambda x: x.std(), 1, acc_z)
    
    # dropping records with NaN values
    df_ = df.dropna(axis=0, how='any')
    if len(df_) != len(df):
        print('Purged %d records containing NaN values, %d remaining' % (len(df)-len(df_), len(df_)))
    
    return {'name': '/'.join(dfraw.Individual.dropna().unique()), 'data': df_}

In [21]:
indiv1 = process_file_tuple(files[0])
print('')
indiv2 = process_file_tuple(files[1])

16293 rows read from ../data/raw/Baen4185_ODBA3D_c.csv
3 duplicate indices removed, 16290 records remaining
16279 rows read from ../data/raw/Baen4185_ODBB.csv
3 duplicate indices removed, 16276 records remaining
Inner-joined dataframes on index, 16276 records remaining
Purged 423 records containing NaN values, 15853 remaining

9172 rows read from ../data/raw/Dorje4014_ODBA3D_c.csv
0 duplicate indices removed, 9172 records remaining
16486 rows read from ../data/raw/Dorje4014_ODBB.csv
0 duplicate indices removed, 16486 records remaining
Inner-joined dataframes on index, 9171 records remaining
Purged 2358 records with less than 40 acc values on at least 1 axis, 6813 remaining
Purged 2544 records containing NaN values, 4269 remaining


In [22]:
indiv1['data']['state'].value_counts()

Rest            10392
Fly              3076
Restless         1835
WHR               308
Others            131
Flap-flight        39
Feed               27
Flap-land          23
Flap-takeoff       22
Name: state, dtype: int64

In [23]:
indiv2['data']['state'].value_counts()

Rest          2719
Fly           1081
Restless       389
Others          45
Flap_fligh      16
WHR             13
Feed             3
Flap_land        2
Flap_takeo       1
Name: state, dtype: int64

In [24]:
for indiv in [indiv1, indiv2]:
    file_name = 'data_clean_' + indiv['name'].replace(' ', '_').replace('(','').replace(')','') + '.csv'
    indiv['data'].to_csv('../data/processed/%s' % file_name, header=True, index=True)
    print('Exported %d records to %s' % (len(indiv['data']), file_name))

Exported 15853 records to data_clean_Thang_Kaar_Baen_4185.csv
Exported 4269 records to data_clean_Thang_Kaar_Dorje_4014.csv
